In [1]:
from socket import * #import the socket module
import time # for RTT
import sys  # forcommandline functions
import datetime
from pathlib import Path

In [2]:
# Import socket module 
import socket 


def Main(): 
    # local host IP '127.0.0.1' 
    host = '127.0.0.1'

    # Define the port on which you want to connect 
    port = 12345

    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 

    # connect to server on local computer 
    s.connect((host,port)) 

    # message you send to server 
    while True: 
        filename = input('\nfile name :') 
        request_type=input('\nrequest type:') 
        # message sent to server 
        s.send((request_type+" "+filename+" "+str(port)).encode('ascii')) 
        if request_type == 'GET':
            message =s.recv(1024).decode()
            if message !="HTTP/1.1 404 file not Found\r\n\r\n":
                with open(filename, 'w+') as f:
                    while message != "\r\n":
                        message =s.recv(1024).decode()
                        f.write(message)
            else:
                print(message)
                break
        elif request_type == 'POST':
            print ('File name is: ', filename)
            my_file = Path(filename)
            if my_file.is_file(): 
                with open(filename, 'r') as f:
                    outputdata = f.read()
                    s.send("HTTP/1.1 200 OK\r\n\r\n".encode())
                    for i in range(0, len(outputdata)):
                        print(outputdata[i])
                        s.send(outputdata[i].encode())
                    s.send("\r\n".encode())
                    s.send("\r\n".encode())
            else:
                s.send(("HTTP/1.1 404 file not Found\r\n\r\n").encode())
                print("HTTP/1.1 404 file not Found\r\n\r\n")
                break
        ans = input('\nDo you want to continue(y/n) :') 
        if ans == 'y': 
            continue
        else: 
            break
    # close the connection 
    s.close() 

if __name__ == '__main__': 
    Main() 




file name :hola.txt

request type:POST
File name is:  hola.txt
H
O
L
A
 
H
O
L
A
 
H
E
R
R
R
R
R
R
R
R
R
E

Do you want to continue(y/n) :y

file name :lala.txt

request type:POST
File name is:  lala.txt
HTTP/1.1 404 file not Found




In [ ]:
from BaseHTTPServer import BaseHTTPRequestHandler
import cgi

class PostHandler(BaseHTTPRequestHandler):
    
    def do_POST(self):
        # Parse the form data posted
        form = cgi.FieldStorage(
            fp=self.rfile, 
            headers=self.headers,
            environ={'REQUEST_METHOD':'POST',
                     'CONTENT_TYPE':self.headers['Content-Type'],
                     })

        # Begin the response
        self.send_response(200)
        self.end_headers()
        self.wfile.write('Client: %s\n' % str(self.client_address))
        self.wfile.write('User-agent: %s\n' % str(self.headers['user-agent']))
        self.wfile.write('Path: %s\n' % self.path)
        self.wfile.write('Form data:\n')

        # Echo back information about what was posted in the form
        for field in form.keys():
            field_item = form[field]
            if field_item.filename:
                # The field contains an uploaded file
                file_data = field_item.file.read()
                file_len = len(file_data)
                del file_data
                self.wfile.write('\tUploaded %s as "%s" (%d bytes)\n' % \
                        (field, field_item.filename, file_len))
            else:
                # Regular form value
                self.wfile.write('\t%s=%s\n' % (field, form[field].value))
        return

if __name__ == '__main__':
    from BaseHTTPServer import HTTPServer
    server = HTTPServer(('localhost', 8080), PostHandler)
    print 'Starting server, use <Ctrl-C> to stop'
    server.serve_forever()

In [ ]:
from BaseHTTPServer import BaseHTTPRequestHandler
import urlparse

class GetHandler(BaseHTTPRequestHandler):
    
    def do_GET(self):
        parsed_path = urlparse.urlparse(self.path)
        message_parts = [
                'CLIENT VALUES:',
                'client_address=%s (%s)' % (self.client_address,
                                            self.address_string()),
                'command=%s' % self.command,
                'path=%s' % self.path,
                'real path=%s' % parsed_path.path,
                'query=%s' % parsed_path.query,
                'request_version=%s' % self.request_version,
                '',
                'SERVER VALUES:',
                'server_version=%s' % self.server_version,
                'sys_version=%s' % self.sys_version,
                'protocol_version=%s' % self.protocol_version,
                '',
                'HEADERS RECEIVED:',
                ]
        for name, value in sorted(self.headers.items()):
            message_parts.append('%s=%s' % (name, value.rstrip()))
        message_parts.append('')
        message = '\r\n'.join(message_parts)
        self.send_response(200)
        self.end_headers()
        self.wfile.write(message)
        return

if __name__ == '__main__':
    from BaseHTTPServer import HTTPServer
    server = HTTPServer(('localhost', 8080), GetHandler)
    print 'Starting server, use <Ctrl-C> to stop'
    server.serve_forever()

In [ ]:
from BaseHTTPServer import HTTPServer, BaseHTTPRequestHandler
from SocketServer import ThreadingMixIn
import threading

class Handler(BaseHTTPRequestHandler):
    
    def do_GET(self):
        self.send_response(200)
        self.end_headers()
        message =  threading.currentThread().getName()
        self.wfile.write(message)
        self.wfile.write('\n')
        return

class ThreadedHTTPServer(ThreadingMixIn, HTTPServer):
    """Handle requests in a separate thread."""

if __name__ == '__main__':
    server = ThreadedHTTPServer(('localhost', 8080), Handler)
    print 'Starting server, use <Ctrl-C> to stop'
    server.serve_forever()

In [ ]:
from BaseHTTPServer import BaseHTTPRequestHandler

class ErrorHandler(BaseHTTPRequestHandler):
    
    def do_GET(self):
        self.send_error(404)
        return

if __name__ == '__main__':
    from BaseHTTPServer import HTTPServer
    server = HTTPServer(('localhost', 8080), ErrorHandler)
    print 'Starting server, use <Ctrl-C> to stop'
    server.serve_forever()